# Structured Output with Ollama

LLM의 응답을 **구조화된 JSON 형식**으로 강제하는 방법을 알아봅니다.

## 일반 응답 vs Structured Output

| 방식 | 출력 형태 | 용도 |
|------|----------|------|
| 일반 응답 | 자유 형식 텍스트 | 대화, 설명 |
| Structured Output | 정해진 JSON 스키마 | API 연동, 데이터 추출 |

## 사전 요구사항
- Ollama 서버 실행 중
- Structured Output을 지원하는 모델 (예: `gpt-oss:20b`, `qwen3`, `llama3.1` 등)

## 1. 환경 설정

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
MODEL_NAME = "gpt-oss:20b"  # Structured Output 지원 모델

print(f"Ollama URL: {OLLAMA_URL}")
print(f"Model: {MODEL_NAME}")

# LangSmith 트레이싱 확인
if os.getenv("LANGCHAIN_API_KEY"):
    print(f"LangSmith: 활성화 ({os.getenv('LANGCHAIN_PROJECT', 'default')})") 
else:
    print("LangSmith: 비활성화")

Ollama URL: http://171.0.53.81:11434
Model: gpt-oss:20b
LangSmith: 활성화 (ai-agent-playground)


In [5]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model=MODEL_NAME,
    base_url=OLLAMA_URL,
    temperature=0,
)

print(f"{MODEL_NAME} 모델 초기화 완료!")

gpt-oss:20b 모델 초기화 완료!


## 2. 일반 응답 (Free-form Text)

In [6]:
# 일반 응답 - 자유 형식 텍스트
response = llm.invoke("서울의 유명한 관광지 3곳을 알려줘")

print("[일반 응답]")
print(response.content)

[일반 응답]
서울에서 꼭 가봐야 할 유명 관광지 3곳을 소개해 드릴게요.

| # | 명소 | 간단한 소개 |
|---|------|-------------|
| 1 | **경복궁 (Gyeongbokgung Palace)** | 조선시대의 대표 궁궐로, 화려한 건축물과 전통적인 궁궐 생활을 체험할 수 있는 곳입니다. 특히, 근정전과 경회루는 꼭 방문해 보세요. |
| 2 | **N서울타워 (N Seoul Tower)** | 남산 정상에 위치한 전망타워로, 서울 전경을 한눈에 볼 수 있는 전망대와 ‘사랑의 자물쇠’가 유명합니다. 밤에는 조명이 아름답게 빛나죠. |
| 3 | **명동 (Myeongdong)** | 쇼핑과 먹거리가 풍부한 번화가로, 한국의 패션 트렌드를 한눈에 볼 수 있고, 길거리 음식(떡볶이, 호떡 등)도 즐길 수 있습니다. |

이 세 곳은 서울을 대표하는 관광지로, 각각 다른 매력을 가지고 있어 방문 시 꼭 한 번씩 경험해 보시길 추천드립니다!


In [7]:
# 프롬프트로 JSON 요청해도 보장되지 않음
response = llm.invoke("서울의 유명한 관광지 3곳을 JSON 형식으로 알려줘")

print("[프롬프트로 JSON 요청]")
print(response.content)
print("\n⚠️ JSON처럼 보이지만, 파싱이 실패할 수 있음 (마크다운, 설명 텍스트 포함 가능)")

[프롬프트로 JSON 요청]
```json
[
  {
    "name": "경복궁",
    "location": "서울특별시 종로구 사직로 161",
    "description": "조선시대의 대표적인 궁궐로, 왕의 생활과 행정이 이루어졌던 곳입니다. 전통 건축물과 아름다운 정원이 인상적이며, 매일 정문 앞에서 근위병 교대식이 진행됩니다."
  },
  {
    "name": "N서울타워",
    "location": "서울특별시 용산구 남산공원로 105",
    "description": "남산 정상에 위치한 전망타워로, 서울 전역을 한눈에 볼 수 있는 전망대와 로맨틱한 사랑의 자물쇠가 유명합니다. 야경이 특히 아름답습니다."
  },
  {
    "name": "명동",
    "location": "서울특별시 중구 명동길",
    "description": "한국의 대표적인 쇼핑 거리로, 화장품, 패션, 음식점이 밀집해 있습니다. 전통과 현대가 공존하는 거리에서 다양한 문화 체험이 가능합니다."
  }
]
```

⚠️ JSON처럼 보이지만, 파싱이 실패할 수 있음 (마크다운, 설명 텍스트 포함 가능)


## 3. Structured Output (JSON 강제)

Pydantic 모델을 사용하여 출력 스키마를 정의하면, 모델이 해당 형식으로만 응답합니다.

In [8]:
from pydantic import BaseModel, Field
from typing import List


# 출력 스키마 정의
class TouristSpot(BaseModel):
    """관광지 정보"""
    name: str = Field(description="관광지 이름")
    description: str = Field(description="간단한 설명")
    recommended_time: str = Field(description="추천 방문 시간")


class TouristSpotList(BaseModel):
    """관광지 목록"""
    city: str = Field(description="도시 이름")
    spots: List[TouristSpot] = Field(description="관광지 목록")


print("스키마 정의 완료!")
print(TouristSpotList.model_json_schema())

스키마 정의 완료!
{'$defs': {'TouristSpot': {'description': '관광지 정보', 'properties': {'name': {'description': '관광지 이름', 'title': 'Name', 'type': 'string'}, 'description': {'description': '간단한 설명', 'title': 'Description', 'type': 'string'}, 'recommended_time': {'description': '추천 방문 시간', 'title': 'Recommended Time', 'type': 'string'}}, 'required': ['name', 'description', 'recommended_time'], 'title': 'TouristSpot', 'type': 'object'}}, 'description': '관광지 목록', 'properties': {'city': {'description': '도시 이름', 'title': 'City', 'type': 'string'}, 'spots': {'description': '관광지 목록', 'items': {'$ref': '#/$defs/TouristSpot'}, 'title': 'Spots', 'type': 'array'}}, 'required': ['city', 'spots'], 'title': 'TouristSpotList', 'type': 'object'}


In [9]:
# Structured Output 적용
structured_llm = llm.with_structured_output(TouristSpotList)

result = structured_llm.invoke("서울의 유명한 관광지 3곳을 알려줘")

print("[Structured Output 결과]")
print(f"타입: {type(result)}")
print(f"\n도시: {result.city}")
print(f"\n관광지 목록:")
for i, spot in enumerate(result.spots, 1):
    print(f"  {i}. {spot.name}")
    print(f"     설명: {spot.description}")
    print(f"     추천 시간: {spot.recommended_time}")

[Structured Output 결과]
타입: <class '__main__.TouristSpotList'>

도시: 서울

관광지 목록:
  1. 경복궁
     설명: 조선시대의 대표 궁궐로, 아름다운 건축물과 전통 문화 체험이 가능한 곳입니다. 왕실 의식이 진행되는 광화문 앞 광장도 꼭 방문해 보세요.
     추천 시간: 오전 10시~오후 3시
  2. N서울타워(남산타워)
     설명: 남산 정상에 위치한 전망타워로, 서울 전경을 한눈에 볼 수 있는 전망대와 로맨틱한 '사랑의 자물쇠'가 유명합니다. 밤에는 조명이 아름답습니다.
     추천 시간: 오후 4시~밤 10시
  3. 명동
     설명: 한국의 대표적인 쇼핑·음식 거리로, 화장품, 패션, 길거리 음식이 풍부합니다. 특히 명동성당과 주변의 전통·현대 건축물도 함께 감상할 수 있습니다.
     추천 시간: 오전 11시~오후 8시


In [10]:
# JSON으로 직렬화
import json

json_output = result.model_dump_json(indent=2, ensure_ascii=False)
print("[JSON 출력]")
print(json_output)

[JSON 출력]
{
  "city": "서울",
  "spots": [
    {
      "name": "경복궁",
      "description": "조선시대의 대표 궁궐로, 아름다운 건축물과 전통 문화 체험이 가능한 곳입니다. 왕실 의식이 진행되는 광화문 앞 광장도 꼭 방문해 보세요.",
      "recommended_time": "오전 10시~오후 3시"
    },
    {
      "name": "N서울타워(남산타워)",
      "description": "남산 정상에 위치한 전망타워로, 서울 전경을 한눈에 볼 수 있는 전망대와 로맨틱한 '사랑의 자물쇠'가 유명합니다. 밤에는 조명이 아름답습니다.",
      "recommended_time": "오후 4시~밤 10시"
    },
    {
      "name": "명동",
      "description": "한국의 대표적인 쇼핑·음식 거리로, 화장품, 패션, 길거리 음식이 풍부합니다. 특히 명동성당과 주변의 전통·현대 건축물도 함께 감상할 수 있습니다.",
      "recommended_time": "오전 11시~오후 8시"
    }
  ]
}


## 4. 다양한 Structured Output 예제

### 4.1 감정 분석

In [11]:
from typing import Literal


class SentimentAnalysis(BaseModel):
    """감정 분석 결과"""
    text: str = Field(description="분석한 원문")
    sentiment: Literal["positive", "negative", "neutral"] = Field(description="감정")
    confidence: float = Field(description="신뢰도 (0.0 ~ 1.0)", ge=0.0, le=1.0)
    reason: str = Field(description="판단 근거")


sentiment_llm = llm.with_structured_output(SentimentAnalysis)

texts = [
    "이 제품 정말 최고예요! 강력 추천합니다.",
    "배송이 너무 늦어서 실망했습니다.",
    "가격은 적당하고 품질은 보통입니다."
]

print("[감정 분석 결과]\n")
for text in texts:
    result = sentiment_llm.invoke(f"다음 텍스트의 감정을 분석해줘: {text}")
    print(f"텍스트: {result.text}")
    print(f"감정: {result.sentiment} (신뢰도: {result.confidence:.2f})")
    print(f"근거: {result.reason}")
    print("-" * 50)

[감정 분석 결과]

텍스트: 이 제품 정말 최고예요! 강력 추천합니다.
감정: positive (신뢰도: 0.98)
근거: 문장은 제품에 대한 극찬과 강력한 추천을 담고 있어, 명백히 긍정적이며 열정적인 감정을 표현합니다.
--------------------------------------------------
텍스트: 배송이 너무 늦어서 실망했습니다.
감정: negative (신뢰도: 0.92)
근거: 문장에 ‘실망’이라는 단어가 직접적으로 부정적 감정을 표현하고 있으며, 배송 지연에 대한 불만이 포함되어 있어 전반적으로 부정적(실망·불만) 감정이 지배적입니다.
--------------------------------------------------
텍스트: 가격은 적당하고 품질은 보통입니다.
감정: neutral (신뢰도: 0.92)
근거: 가격이 ‘적당하다’는 표현은 긍정적이지만, ‘품질이 보통이다’는 중립적이거나 다소 부정적일 수 있는 표현입니다. 두 요소를 종합하면 전반적으로 감정이 크게 기울어지지 않아 ‘중립’이라고 판단됩니다.
--------------------------------------------------


### 4.2 정보 추출 (Named Entity Recognition)

In [12]:
from typing import Optional


class Person(BaseModel):
    """인물 정보"""
    name: str = Field(description="이름")
    role: Optional[str] = Field(description="직책/역할", default=None)
    organization: Optional[str] = Field(description="소속 조직", default=None)


class ExtractedInfo(BaseModel):
    """추출된 정보"""
    people: List[Person] = Field(description="언급된 인물들")
    organizations: List[str] = Field(description="언급된 조직들")
    locations: List[str] = Field(description="언급된 장소들")
    dates: List[str] = Field(description="언급된 날짜들")


extraction_llm = llm.with_structured_output(ExtractedInfo)

news = """
삼성전자 이재용 회장은 2024년 1월 15일 서울 서초구 삼성타운에서 
마이크로소프트 사티아 나델라 CEO와 만나 AI 반도체 협력 방안을 논의했다.
양측은 향후 실리콘밸리에서 추가 회동을 가질 예정이다.
"""

result = extraction_llm.invoke(f"다음 뉴스에서 정보를 추출해줘:\n{news}")

print("[정보 추출 결과]\n")
print("인물:")
for p in result.people:
    print(f"  - {p.name} ({p.role}, {p.organization})")
print(f"\n조직: {result.organizations}")
print(f"장소: {result.locations}")
print(f"날짜: {result.dates}")

[정보 추출 결과]

인물:
  - 이재용 (회장, 삼성전자)
  - 사티아 나델라 (CEO, 마이크로소프트)

조직: ['삼성전자', '마이크로소프트']
장소: ['서울 서초구 삼성타운']
날짜: ['2024-01-15']


### 4.3 분류 (Classification)

In [13]:
class TicketClassification(BaseModel):
    """고객 문의 분류"""
    category: Literal["기술지원", "결제문의", "배송문의", "환불요청", "기타"] = Field(
        description="문의 카테고리"
    )
    priority: Literal["높음", "보통", "낮음"] = Field(description="우선순위")
    summary: str = Field(description="문의 요약 (1문장)")
    suggested_action: str = Field(description="권장 조치")


ticket_llm = llm.with_structured_output(TicketClassification)

tickets = [
    "앱이 자꾸 튕겨요. 아이폰 15에서 사용중인데 로그인하면 바로 꺼집니다.",
    "결제했는데 포인트가 안 들어왔어요. 주문번호 12345입니다.",
    "일주일 전에 주문했는데 아직도 배송 시작도 안 됐네요?"
]

print("[문의 분류 결과]\n")
for ticket in tickets:
    result = ticket_llm.invoke(f"다음 고객 문의를 분류해줘:\n{ticket}")
    print(f"문의: {ticket[:50]}...")
    print(f"카테고리: {result.category} | 우선순위: {result.priority}")
    print(f"요약: {result.summary}")
    print(f"권장 조치: {result.suggested_action}")
    print("-" * 60)

[문의 분류 결과]

문의: 앱이 자꾸 튕겨요. 아이폰 15에서 사용중인데 로그인하면 바로 꺼집니다....
카테고리: 기술지원 | 우선순위: 높음
요약: 앱이 로그인 시 바로 충돌(앱 충돌)
권장 조치: 앱 재설치, 최신 버전 확인, 로그 수집 후 개발팀에 전달
------------------------------------------------------------
문의: 결제했는데 포인트가 안 들어왔어요. 주문번호 12345입니다....
카테고리: 결제문의 | 우선순위: 보통
요약: 포인트 적립이 안 됨
권장 조치: 포인트 적립 여부 확인 및 고객에게 안내
------------------------------------------------------------
문의: 일주일 전에 주문했는데 아직도 배송 시작도 안 됐네요?...
카테고리: 배송문의 | 우선순위: 높음
요약: 주문한 상품이 아직 배송이 시작되지 않아 배송 지연에 대한 문의입니다.
권장 조치: 배송 상태 확인 및 고객에게 예상 배송일 안내
------------------------------------------------------------


## 5. Tool Calling과의 비교

| 기능 | Structured Output | Tool Calling |
|------|-------------------|---------------|
| 목적 | 응답 형식 강제 | 외부 함수 호출 |
| 출력 | Pydantic 객체 | 함수 호출 JSON |
| 실행 | 없음 (데이터만) | 함수 실행 필요 |
| 용도 | 데이터 추출, 분류 | 검색, 계산, API 호출 |

In [ ]:
# Structured Output: 데이터 추출만
class WeatherRequest(BaseModel):
    """날씨 요청 정보 추출"""
    city: str = Field(description="도시 이름")
    date: Optional[str] = Field(description="날짜 (없으면 오늘)", default=None)


weather_parser = llm.with_structured_output(WeatherRequest)
parsed = weather_parser.invoke("내일 부산 날씨 어때?")

print("[Structured Output]")
print(f"추출된 정보: city={parsed.city}, date={parsed.date}")
print("→ 실제 날씨 API 호출은 별도로 해야 함")

In [ ]:
# Tool Calling: 함수 실행 의도
from langchain_core.tools import tool

@tool
def get_weather(city: str, date: str = "today") -> str:
    """도시의 날씨를 가져옵니다."""
    return f"{city}의 {date} 날씨: 맑음, 15°C"


llm_with_tools = llm.bind_tools([get_weather])
response = llm_with_tools.invoke("내일 부산 날씨 어때?")

print("[Tool Calling]")
if response.tool_calls:
    tc = response.tool_calls[0]
    print(f"호출할 함수: {tc['name']}({tc['args']})")
    print("→ 이 정보로 실제 함수를 실행해야 함")

## 6. 요약

### Structured Output 사용 시점
- 응답을 프로그래밍적으로 처리해야 할 때
- 일관된 JSON 형식이 필요할 때
- 데이터 추출, 분류, 분석 작업

### 주요 메서드
```python
# Pydantic 모델로 출력 강제
structured_llm = llm.with_structured_output(MySchema)
result = structured_llm.invoke("질문")

# result는 MySchema 인스턴스
result.field_name  # 직접 접근 가능
result.model_dump_json()  # JSON 문자열로 변환
```

### 지원 모델
Structured Output을 지원하려면 모델이 JSON 형식 출력을 학습했어야 합니다:
- `gpt-oss:20b` ✅
- `qwen3` 시리즈 ✅
- `llama3.1`, `llama3.2` ✅
- `mistral` ✅